In [4]:
import cv2
import numpy as np
import glob
from skimage.segmentation import slic, find_boundaries
from skimage.color import label2rgb

In [5]:
def classify_pixels(original_image_path, mask_image_path):
    # Load images
    original_image = cv2.imread(original_image_path, cv2.IMREAD_COLOR)
    original_image = cv2.resize(original_image, (256, 256))
    mask_image = cv2.imread(mask_image_path, cv2.IMREAD_GRAYSCALE)
    mask_image = cv2.resize(mask_image, (256, 256))

    # Prepare an output image with the same dimensions and type as the mask
    output_image = np.zeros_like(mask_image)

    # Get the dimensions of the image
    rows, cols = mask_image.shape

    # Iterate over each pixel in the mask
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            # Current pixel and its 8 neighbors
            current_pixel = mask_image[i, j]
            neighborhood = mask_image[i-1:i+2, j-1:j+2]

            # Check if all neighbors are the same as the current pixel
            if np.all(neighborhood == current_pixel):
                # If current pixel is foreground and all neighbors are also foreground
                if current_pixel == 255:
                    output_image[i, j] = 255  # White
                else:
                    output_image[i, j] = 128  # Gray for background
            else:
                output_image[i, j] = 0  # Black for boundary

    return output_image

# Paths (mocked paths for demonstration; replace these with your actual paths)
original_image_paths = glob.glob('/home/somayeh/PycharmProjects/superpixel_segmentation/boundary_pixels/skin/original_image/*.png')
mask_image_paths = glob.glob('/home/somayeh/PycharmProjects/superpixel_segmentation/boundary_pixels/skin/original_mask/*.png')

# # Process each pair of image and mask
# for original_path, mask_path in zip(original_image_paths, mask_image_paths):
#     result_image = classify_pixels(original_path, mask_path)
#     # Save or display result_image as needed
#     Example: cv2.imshow('Result', result_image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
for original_path, mask_path in zip(original_image_paths, mask_image_paths):
    result_image = classify_pixels(original_path, mask_path)
    # Construct a save path for the result image
    save_path = original_path.replace('original_image', 'processed_results').replace('.png', '_result.png')
    cv2.imwrite(save_path, result_image)  # Save the processed image

In [ ]:
def classify_and_superpixels(original_image_path, mask_image_path):
    # Load images
    original_image = cv2.imread(original_image_path, cv2.IMREAD_COLOR)
    mask_image = cv2.imread(mask_image_path, cv2.IMREAD_GRAYSCALE)

    # Process the mask image to get the classified image
    output_image = np.zeros_like(mask_image)

    rows, cols = mask_image.shape
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            current_pixel = mask_image[i, j]
            neighborhood = mask_image[i-1:i+2, j-1:j+2]

            if np.all(neighborhood == current_pixel):
                if current_pixel == 255:
                    output_image[i, j] = 255  # White
                else:
                    output_image[i, j] = 128  # Gray
            else:
                output_image[i, j] = 0  # Black

    # Apply SLIC superpixel segmentation
    segments = slic(original_image, n_segments=1200, compactness=10, sigma=1)

    # Create a boundary map from SLIC segments
    boundaries = find_boundaries(segments, mode='thin')

    # Convert output image to color (3 channels) for visualization
    colored_output = cv2.cvtColor(output_image, cv2.COLOR_GRAY2BGR)

    # Draw boundaries: set to blue color
    colored_output[boundaries] = [255, 0, 0]  # Blue

    return colored_output

# Paths
original_image_paths = glob.glob('/home/somayeh/PycharmProjects/superpixel_segmentation/best_worst/chest/original_image/*.png')
mask_image_paths = glob.glob('/home/somayeh/PycharmProjects/superpixel_segmentation/best_worst/chest/original_mask/*.png')

# Process each pair of image and mask
for original_path, mask_path in zip(original_image_paths, mask_image_paths):
    result_image = classify_and_superpixels(original_path, mask_path)
    # Here you can save or display the result image
    cv2.imshow('Superpixel Classifications', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()